Created 6/21/2023

The purpose of this notebook is to push towards getting the final assembly of the simulated fields working. It will only call from the .py file versions of functions and classes, and will attempt to demonstrate full functionality from the beginning to the end of created a simulated data product.


In [1]:
# imports

In [2]:
# download data

In [ ]:
# set up the generator classes and the function selector

In [4]:
# assemble field
# probably need to add in stand ins for the empirical noise model and the LC pipeline

# later add in something to pick the field

In [ ]:
# test field